In [1]:
from pathlib import Path
import fiona

Path('src/st_preprocessing/locations/sources/lion.py')

from st_preprocessing.locations.sources.lion import LocationsLIONLoader, LION_PATH
fiona.listlayers(LION_PATH)

['node', 'node_stname', 'altnames', 'lion']

In [2]:
# fiona
# lion_layers = LocationsLIONLoader._load_baselayers()
import geopandas as gpd
node = gpd.read_file(LION_PATH, layer='node')
node_stname = gpd.read_file(LION_PATH, layer='node_stname')
lion = gpd.read_file(LION_PATH, layer='lion')


In [22]:
# DATA DICTIONARIES
lion['FeatureTyp'].value_counts()

FeatureTypDict = {
    '0': 'Street',
    '1': 'Railroad',
    '2': 'Shoreline',
    '3': 'CB Boundary',
    '5': 'Paper Street',
    '6': 'Private Street',
    '7': 'Distict Boundary',
    '8': 'Physical Non-Street Boundary',
    '9': 'Paper Street and Boundary',
    'A': 'Alley',
    'W': 'Non-Vehicular Path',
    'C': 'Corporation Counsel Opinion',
    'F': 'Ferry Route'
}

SegmentTypDict = {
    'B': 'Both Generic and Roadbed',
    'C': 'Connector: Connects adacent roadbed segments of dividied roadway',
    'E': 'Entrance/Exit Ramp',
    'F': 'Faux Segment',
    'G': 'Generic Segment',
    'R': 'Roadbed Segment: part of physically divided street',
    'T': 'Terminator',
    'U': 'Unidivided Street',
    'S': 'Suppressed'
}

BikeLaneDict = {
    '1': 'Class I: Separated Greenway',
    '2': 'Class II: Striped Bike Lane',
    '3': 'Class III: Signed Bicycle Route',
    '4': 'Links: Connecting Segments',
    '5': 'Class I, II',
    '6': 'Class II, III',
    '7': 'Stairs',
    '8': 'Class I, III', 
    '9': 'Class II, I',
    '10': 'Class III, I',
    '11': 'Class III, II'
}

lion['SegmentTyp'].value_counts()

import json


# with open(Path(os.getcwd()).parent / 'src' / 'st_preprocessing' / 'locations' / 'sources' / 'data_dictionaries' / 'lion.json','w+') as f: 
#     datadict = {
#         'BikeLane': BikeLaneDict,
#         'SegmentTyp': SegmentTypDict,
#         'FeatureTyp': FeatureTypDict
#     }
#     f.write(json.dumps(datadict))
import os
Path(os.getcwd()).parent / 'src' / 'st_preprocessing' / 'locations' / 'sources' / 'data_dictionaries'

In [26]:
import osmnx as ox
import pandas as pd
# Masks to filter LION
mask_roads = lion['FeatureTyp'].isin(['0']) # Mask only the roads
mask_road_paths = lion['FeatureTyp'].isin(['0', 'W']) # Also mask the walking paths

active_nodes = pd.concat([lion.loc[mask_road_paths,'NodeIDFrom'], lion.loc[mask_road_paths, 'NodeIDTo']]).unique().astype(int)

lion_road_edges = lion[mask_roads][['Street','FeatureTyp','NodeIDTo', 'NodeIDFrom', 'SegmentTyp', 'TrafDir', 'geometry']].to_crs('4326')
lion_road_nodes = node[node['NODEID'].isin(active_nodes)].to_crs('4326')


,OBJECTID,NODEID,GLOBALID,VIntersect,geometry
8,9,9,{28CA2474-1450-4EDF-ACBA-5A743CFF4CF6},,POINT (-74.25411 40.50234)
9,10,10,{2359B033-9EA5-483C-8352-F2D73159FB67},,POINT (-74.25379 40.50239)
10,11,11,{F55DF044-C4A7-4E1A-B5ED-25A3C1721F43},,POINT (-74.25273 40.50252)
12,13,13,{4789EEA6-1336-48DB-A051-C81850BE6A37},,POINT (-74.25433 40.50531)
13,14,14,{FED57E01-83E6-4ECA-B4B9-99324F27F852},,POINT (-74.25422 40.50617)
...,...,...,...,...,...
139387,869929,9067040,{C40D80C3-6EA8-4025-9FF7-57E12CF192DD},,POINT (-73.8876 40.65156)
139388,869930,9067041,{2B376C58-699A-40C0-80CD-AED6CFF4E4D0},,POINT (-73.8873 40.65107)
139389,869959,9067044,{01FF2A76-2694-4C2E-AEFB-3CDFC945B57A},,POINT (-73.96552 40.6935)
139390,870247,9067045,{A1B1CB64-C0B7-4DDB-9BF7-714BAC26A819},,POINT (-74.15186 40.54982)


In [52]:
# #lion['BikeLane'].value_counts() # This gives us actual classes
# ox.graph_from_gdfs(
#     graph_nodes = lion_road_nodes.set_index('NODEID'),
#     graph_edges = lion_road_nodes.set_index('NODEID'),
# )

lion_road_edges.explode('geometry').set_index(['NodeIDTo','NodeIDFrom'])

Street FeatureTyp SegmentTyp TrafDir  \
NodeIDTo NodeIDFrom                                                  
9045677  0047740     EAST 168 STREET          0          U       T   
0048678  0048679     WEST 192 STREET          0          U       A   
0047822  0047288        UNION AVENUE          0          U       W   
         0047288        UNION AVENUE          0          U       W   
         0047288        UNION AVENUE          0          U       W   
...                              ...        ...        ...     ...   
9066987  9066986         FONDA PLACE          0          U       T   
9067045  0004353        NOLAN AVENUE          0          U       A   
0004354  9067045        NOLAN AVENUE          0          U       A   
9067046  0002643       WILSON AVENUE          0          U       W   
0002550  9067046       WILSON AVENUE          0          U       W   

                                                              geometry  
NodeIDTo NodeIDFrom                                                     
9045677  0047740     LINESTRING (-73.90347 40.83035, -73.90238 40.8...  
0048678  0048679      LINESTRING (-73.9012 40.86661, -73.90207 40.867)  
0047822  0047288     LINESTRING (-73.90118 40.82439, -73.90051 40.8...  
         0047288     LINESTRING (-73.90118 40.82439, -73.90051 40.8...  
         0047288     LINESTRING (-73.90118 40.82439, -73.90051 40.8...  
...                                                                ...  
9066987  9066986     LINESTRING (-74.21097 40.52897, -74.21022 40.5...  
9067045  0004353     LINESTRING (-74.15207 40.55036, -74.15186 40.5...  
0004354  9067045     LINESTRING (-74.15186 40.54982, -74.15183 40.5...  
9067046  0002643     LINESTRING (-74.16574 40.5454, -74.16618 40.54...  
0002550  9067046     LINESTRING (-74.16618 40.54524, -74.16667 40.5...  

[188843 rows x 5 columns]

In [47]:
G = ox.graph_from_place('Piedmont, CA, USA')
nodes, edges = ox.graph_to_gdfs(G)

#nodes.rename(index={'osmid':'nodeid'}, axis)
nodes.index#.rename({'osmid': 'NODEID'})
nodes.index.name = 'NODEID'
G_recreated = ox.graph_from_gdfs(nodes, edges)
G_recreated

In [ ]:
#ox.graph_to_gdfs(G_recreated)


,y,x,street_count,highway,junction,geometry
NODEID,,,,,,
53017091,37.826250,-122.247604,4,NaN,NaN,POINT (-122.2476 37.82625)
53018397,37.824220,-122.247198,3,stop,NaN,POINT (-122.2472 37.82422)
53018399,37.824900,-122.246113,3,NaN,NaN,POINT (-122.24611 37.8249)
53018402,37.825406,-122.245338,3,NaN,NaN,POINT (-122.24534 37.82541)
53018404,37.825948,-122.244467,3,NaN,NaN,POINT (-122.24447 37.82595)
...,...,...,...,...,...,...
12420966184,37.822813,-122.239492,3,NaN,NaN,POINT (-122.23949 37.82281)
12420966187,37.822737,-122.239373,4,crossing,NaN,POINT (-122.23937 37.82274)
13171562969,37.828869,-122.228689,1,NaN,NaN,POINT (-122.22869 37.82887)


In [ ]:

#ox.graph_from_gdfs(gdf_nodes=lion_road_nodes, gdf_edges=lion_road_edges, 

ValueError: `gdf_nodes` must contain 'x' and 'y' columns.

In [ ]:


#node[~node['NODEID'].isin(active_nodes)].plot()#.value_counts()
#active_nodes in node['NODEID']
#node['NODEID']

array([  47740,   48679,   47288, ..., 9066763, 9066891, 9066987],
      shape=(116877,))